<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9f506ad953ef2fd213e50ed7a80b5446184fc740b276123ce4e1b4fe690e9bb3
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-16 15:19:07
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: 92.83 K (0.67%)
Current PnL: -21.08 L (-14.04%)
CY Booked + Current PnL: -9.18 L (-6.11%)
-------------------
Total profit:  1.79 L
Total loss:  -22.87 L
-------------------
Total Booked + Current PnL: 17.80 L (14.46%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.48%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.27 L (62.86%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.82%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.44,59.0,X-LC,3.79,215104.0,10604.0,12691.0,0.73,5.19,5.90,11.39,10.0,0.84,1.54,14.60,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,100.43,58.0,M-SC,1.56,86329.0,-14448.0,14538.0,2.42,-14.34,16.84,0.09,245.0,-0.99,0.62,13.42,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.16,58.0,X-MC,1.27,212370.0,20628.0,17138.0,0.26,10.76,8.07,19.69,99.0,1.20,1.52,18.29,XY25,NTT,AC
33,ICICIGI,2260.25,-13.38,66.0,X-MC,2.07,148821.0,12688.0,18439.0,-0.46,9.32,12.39,22.86,91.0,0.69,1.07,23.11,X40,ATH,INSURANCE
41,ITC,452.00,-38.77,52.0,X-LC,2.87,198192.0,-1946.0,22832.0,1.35,-0.97,11.52,10.44,4.0,-0.09,1.42,4.54,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-44.96,52.0,M-SC,4.23,93068.0,1570.0,77181.0,4.89,1.72,82.93,86.07,223.0,0.02,0.67,35.15,XR,NTT,DURABLES
49,NESTLEIND,1377.00,-2.88,72.0,X-LC,2.62,298865.0,33439.0,23341.0,4.57,12.60,7.81,21.40,11.0,1.43,2.15,20.08,XY25,NTT,FMCG
82,VBL,671.64,-18.24,54.0,X-LC,10.58,294278.0,-21564.0,134220.0,3.57,-6.83,45.61,35.67,5.0,-0.16,2.11,6.08,X40N,ATH,FMCG
66,SONACOMS,1006.00,-34.18,68.0,M-SC,12.29,81926.0,-19233.0,94125.0,3.52,-19.01,114.89,74.03,202.0,-0.20,0.59,15.52,AR,BTT,AUTO
63,SHALBY,327.00,1190.27,56.0,M-SC,23.02,175433.0,-5710.0,51174.0,3.39,-3.15,29.17,25.10,235.0,-0.11,1.26,40.59,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.00,-280.91,30.0,H-SC,4.20,100470.0,-63135.0,184453.0,-1.94,-38.59,183.59,74.15,139.0,-0.34,0.72,28.79,XR,NTT,CHEMICALS
27,HATHWAY,31.40,852.90,36.0,H-SC,5.20,82140.0,-26580.0,106256.0,-1.93,-24.45,129.36,73.29,161.0,-0.25,0.59,9.87,XR,NTT,ENTERTAINMENT
50,PGHH,17907.65,-27.11,54.0,X-MC,3.24,210930.0,10110.0,57689.0,-1.51,5.03,27.35,33.76,80.0,0.18,1.51,10.49,X40,ATH,FMCG
53,RAJOOENG,143.10,-43.55,39.0,H-SC,17.77,85500.0,-17000.0,57601.0,-1.30,-16.59,67.37,39.61,114.0,-0.30,0.61,4.42,AR,ATH,MISC
31,HINDZINC,730.22,34.91,65.0,M-LC,9.46,220371.0,15295.0,97272.0,-1.29,7.46,44.14,54.89,52.0,0.16,1.58,33.35,X5K,ATH,METALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-186.36,69.0,M-SC,17.64,150075.0,855.0,143322.0,1.07,0.57,95.5,96.62,208.0,0.01,1.08,62.41,XY25,NTT,FINANCE
59,SAIL,228.0,43.54,50.0,M-MC,12.22,226288.0,1326.0,166322.0,0.86,0.59,73.5,74.53,192.0,0.01,1.62,33.03,XY24,BTT,STEEL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,108.87,62.0,H-SC,11.94,132300.0,-10179.0,29701.0,0.89,-7.14,22.45,13.70,163.0,-0.34,0.95,56.15,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,100.43,58.0,M-SC,1.56,86329.0,-14448.0,14538.0,2.42,-14.34,16.84,0.09,245.0,-0.99,0.62,13.42,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,1.91,220869.0,-48798.0,85123.0,-0.10,-18.10,38.54,13.47,138.0,-0.57,1.59,12.31,XY24,NTT,PAINTS
17,CERA,9475.0,-23.87,40.0,H-SC,2.21,138920.0,-36983.0,79004.0,-0.21,-21.02,56.87,23.89,149.0,-0.47,1.00,19.98,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.05,46.0,H-MC,7.14,102686.0,-29149.0,71377.0,0.69,-22.11,69.51,32.03,98.0,-0.41,0.74,15.02,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,52.0,M-LC,6.14,207488.0,4318.0,37804.0,0.39,2.13,18.22,20.74,55.0,0.11,1.49,7.79,XY25,NTT,FINANCE
59,SAIL,228.00,43.54,50.0,M-MC,12.22,226288.0,1326.0,166322.0,0.86,0.59,73.50,74.53,192.0,0.01,1.62,33.03,XY24,BTT,STEEL
31,HINDZINC,730.22,34.91,65.0,M-LC,9.46,220371.0,15295.0,97272.0,-1.29,7.46,44.14,54.89,52.0,0.16,1.58,33.35,X5K,ATH,METALS
85,WIPRO,420.00,-10.62,61.0,M-LC,5.77,157455.0,6510.0,102944.0,1.50,4.31,65.38,72.51,53.0,0.06,1.13,10.60,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,54.0,M-LC,8.73,157759.0,16919.0,27576.0,-0.49,12.01,17.48,31.59,66.0,0.61,1.13,41.68,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,34.91,65.0,M-LC,9.46,220371.0,15295.0,97272.0,-1.29,7.46,44.14,54.89,52.0,0.16,1.58,33.35,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,60.0,H-MC,13.00,192850.0,13042.0,99376.0,-0.75,7.25,51.53,62.52,88.0,0.13,1.38,43.81,XR,NTT,BANKS
85,WIPRO,420.00,-10.62,61.0,M-LC,5.77,157455.0,6510.0,102944.0,1.50,4.31,65.38,72.51,53.0,0.06,1.13,10.60,XR,NTT,IT
84,WHIRLPOOL,2270.00,-44.96,52.0,M-SC,4.23,93068.0,1570.0,77181.0,4.89,1.72,82.93,86.07,223.0,0.02,0.67,35.15,XR,NTT,DURABLES
36,INDIAMART,4810.62,-55.20,40.0,H-SC,8.07,124889.0,1553.0,130072.0,1.15,1.26,104.15,106.72,119.0,0.01,0.90,24.93,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,5.42,144020.0,-132503.0,242573.0,1.51,-47.92,168.43,39.81,54.0,-0.55,1.03,1.51,XY24,NTT,AUTO
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52133.0,-12873.0,163802.0,-0.10,-19.80,314.20,232.18,198.0,-0.08,0.37,1.67,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.17,39.0,X-MC,1.82,230256.0,-17475.0,92540.0,0.81,-7.05,40.19,30.30,92.0,-0.19,1.65,5.30,X40,ATH,ELECTRICAL
34,ICICIPRULI,790.00,-22.83,40.0,X-MC,2.20,132696.0,-3092.0,45846.0,-0.14,-2.28,34.55,31.48,107.0,-0.07,0.95,9.57,X40,ATH,INSURANCE
12,BATAINDIA,2096.00,-36.29,41.0,X-SC,4.33,95242.0,-33428.0,78727.0,2.12,-25.98,82.66,35.20,219.0,-0.42,0.68,9.32,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.04,45.0,H-LC,0.62,155995.0,-30100.0,77576.0,0.04,-16.17,49.73,25.51,15.0,-0.39,1.12,14.96,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,1.16,58.0,X-MC,1.27,212370.0,20628.0,17138.0,0.26,10.76,8.07,19.69,99.0,1.20,1.52,18.29,XY25,NTT,AC
28,HAVELLS,2069.16,-7.17,39.0,X-MC,1.82,230256.0,-17475.0,92540.0,0.81,-7.05,40.19,30.30,92.0,-0.19,1.65,5.30,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.38,66.0,X-MC,2.07,148821.0,12688.0,18439.0,-0.46,9.32,12.39,22.86,91.0,0.69,1.07,23.11,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-2.88,72.0,X-LC,2.62,298865.0,33439.0,23341.0,4.57,12.60,7.81,21.40,11.0,1.43,2.15,20.08,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,5.42,144020.0,-132503.0,242573.0,1.51,-47.92,168.43,39.81,54.0,-0.55,1.03,1.51,XY24,NTT,AUTO
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52133.0,-12873.0,163802.0,-0.10,-19.80,314.20,232.18,198.0,-0.08,0.37,1.67,XY24,NTT,MISC
73,TCS,4389.97,-29.97,45.0,X-LC,12.48,279236.0,-66720.0,133419.0,0.03,-19.29,47.78,19.28,1.0,-0.50,2.00,3.23,X40,ATH,IT
9,BAJAJHFL,181.50,-14.48,45.0,X-MC,5.81,180416.0,-20090.0,117252.0,-0.11,-10.02,64.99,48.45,90.0,-0.17,1.30,4.04,X40N,ATH,FINANCE
2,ACC,3906.00,-37.64,52.0,X-MC,3.36,185790.0,-52161.0,204815.0,0.03,-21.92,110.24,64.15,174.0,-0.25,1.33,4.22,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52133.0,-12873.0,163802.0,-0.10,-19.80,314.20,232.18,198.0,-0.08,0.37,1.67,XY24,NTT,MISC
55,RELAXO,1176.00,-39.30,44.0,X-SC,4.00,82767.0,-62393.0,138320.0,-0.40,-42.98,167.12,52.31,136.0,-0.45,0.59,10.79,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.29,41.0,X-SC,4.33,95242.0,-33428.0,78727.0,2.12,-25.98,82.66,35.20,219.0,-0.42,0.68,9.32,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.61,48.0,X-SC,1.02,107226.0,5222.0,38709.0,-0.17,5.12,36.10,43.06,122.0,0.13,0.77,23.11,X40N,ATH,MISC
32,HONAUT,58357.33,-23.25,44.0,X-SC,2.40,108225.0,-19713.0,66851.0,-0.22,-15.41,61.77,36.84,143.0,-0.29,0.78,10.80,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.97,45.0,X-LC,12.48,279236.0,-66720.0,133419.0,0.03,-19.29,47.78,19.28,1.0,-0.50,2.00,3.23,X40,ATH,IT
39,INFY,2275.00,-19.60,47.0,X-LC,7.54,313877.0,989.0,170686.0,-0.05,0.32,54.38,54.87,3.0,0.01,2.25,6.99,X40,BTT,IT
41,ITC,452.00,-38.77,52.0,X-LC,2.87,198192.0,-1946.0,22832.0,1.35,-0.97,11.52,10.44,4.0,-0.09,1.42,4.54,X40,NTT,FMCG
82,VBL,671.64,-18.24,54.0,X-LC,10.58,294278.0,-21564.0,134220.0,3.57,-6.83,45.61,35.67,5.0,-0.16,2.11,6.08,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,51.0,H-LC,7.24,250104.0,-11515.0,130729.0,0.72,-4.40,52.27,45.57,7.0,-0.09,1.80,6.42,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6761.11,58.0,L-SC,6.17,75262.0,-18548.0,95169.0,2.21,-19.77,126.45,81.67,269.0,-0.19,0.54,45.75,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,51.0,L-SC,28.78,82042.0,-31507.0,71557.0,0.28,-27.75,87.22,35.27,268.0,-0.44,0.59,100.80,XR,NTT,HOTELS
48,MASFIN,398.61,-20.05,45.0,H-SC,9.48,90870.0,-7110.0,28715.0,0.60,-7.26,31.60,22.05,152.0,-0.25,0.65,32.09,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,108.87,62.0,H-SC,11.94,132300.0,-10179.0,29701.0,0.89,-7.14,22.45,13.70,163.0,-0.34,0.95,56.15,OX40N,NTT,BANKS
68,SURYODAY,240.00,55.70,52.0,H-SC,10.24,143534.0,-35537.0,101263.0,-0.95,-19.85,70.55,36.71,135.0,-0.35,1.03,42.41,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,17.64,150075.0,855.0,143322.0,1.07,0.57,95.50,96.62,208.0,0.01,1.08,62.41,XY25,NTT,FINANCE
4,ANGELONE,3033.00,14.80,69.0,X-SC,9.38,203524.0,12518.0,45182.0,1.50,6.55,22.20,30.21,157.0,0.28,1.46,28.80,X40N,NTT,FINANCE
63,SHALBY,327.00,1190.27,56.0,M-SC,23.02,175433.0,-5710.0,51174.0,3.39,-3.15,29.17,25.10,235.0,-0.11,1.26,40.59,XY24,NTT,HEALTHCARE
68,SURYODAY,240.00,55.70,52.0,H-SC,10.24,143534.0,-35537.0,101263.0,-0.95,-19.85,70.55,36.71,135.0,-0.35,1.03,42.41,XR,NTT,BANKS
31,HINDZINC,730.22,34.91,65.0,M-LC,9.46,220371.0,15295.0,97272.0,-1.29,7.46,44.14,54.89,52.0,0.16,1.58,33.35,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.63
1,25,44.10
2,50,75.87


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.36
LC    33.81
MC    21.80
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.57
X40      14.76
XY25     12.29
XR       10.94
X40N     10.23
AR        8.18
OX40N     7.64
X200      1.82
X5K       1.58
SR        1.13
MH        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.46
X-LC    22.71
X-MC    16.08
M-SC    12.78
M-LC     5.33
H-LC     4.74
X-SC     4.65
H-MC     3.80
M-MC     1.62
L-SC     1.47
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.23,-3.51,37.62
IT,12.94,-16.61,79.68
FINANCE,12.22,-11.06,61.41
MISC,6.85,-20.31,86.98
BANKS,6.34,-12.78,72.11
PAINTS,5.58,-18.89,36.61
ELECTRICAL,5.35,-10.58,49.38
INSURANCE,3.94,-0.86,38.56
AUTO,2.85,-44.93,114.21


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3312588.0,22
XR,1384623.0,14
AR,1191087.0,9
X40,779510.0,10
X40N,602396.0,8
OX40N,582392.0,10
XY25,471417.0,8
SR,264391.0,2
X5K,97272.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3454388.0,25
M-SC,1262122.0,15
X-MC,1085299.0,11
X-LC,1073565.0,13
X-SC,531591.0,6
H-MC,349738.0,3
H-LC,280547.0,3
M-LC,265596.0,4
L-SC,256077.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1196776.0      6
           XR         828689.0      7
           AR         821615.0      5
M-SC       XY24       683874.0      6
X-MC       XY24       588003.0      4
X-LC       X40        486269.0      5
           XY24       307250.0      2
X-SC       X40N       289062.0      4
H-SC       OX40N      274361.0      4
           SR         264391.0      2
X-MC       X40        214514.0      4
H-LC       AR         208305.0      2
X-LC       X40N       196082.0      3
H-MC       XY24       178985.0      1
L-SC       XR         166726.0      2
M-MC       XY24       166322.0      1
X-MC       XY25       165530.0      2
X-SC       XY24       163802.0      1
M-SC       AR         161167.0      2
           OX40N      147303.0      4
           XY25       143322.0      1
           XR         126456.0      2
X-MC       X40N       117252.0      1
M-LC       XR         102944.0      1
H-MC       XR          99376.0      1
M-LC       X5K         97272.0      1
L-SC       OX40N       89351.0      1
X-LC       XY25        83964.0      3
X-SC       X40         78727.0      1
H-LC       X200        72242.0      1
H-MC       OX40N       71377.0      1
H-SC       MH          68556.0      1
L-MC       XR          60432.0      1
L-LC       XY25        40797.0      1
M-LC       XY25        37804.0      1
           XY24        27576.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
